In [959]:
import pandas as pd
import numpy as np
import os
import re

In [960]:
base_dir = os.getcwd()
df1 = pd.read_csv(base_dir+'/data.csv', names=['name', 'price', 'score', 'temp'])
df2 = pd.read_csv(base_dir+'/data2.csv', names=['name', 'price', 'score', 'temp'])

In [961]:
df1.head()

,name,price,score,temp
0,Larose Red Blend 2019,"₩286,618",4.7,"['width: 15%; left: 66.7221%;', 'width: 15%; l..."
1,Elevage 2000,"₩360,000",4.7,"['width: 15%; left: 84.3588%;', 'width: 15%; l..."
2,Spring Mountain Cabernet Sauvignon 2011,"₩500,000",4.7,"['width: 15%; left: 85%;', 'width: 15%; left: ..."
3,Spring Mountain Cabernet Sauvignon 2016,"₩500,000",4.6,"['width: 15%; left: 85%;', 'width: 15%; left: ..."
4,Kai Carmenère 2008,"₩500,000",4.6,"['width: 15%; left: 74.4108%;', 'width: 15%; l..."


In [962]:
df2.head()

,name,price,score,temp
0,S.L.V. Cabernet Sauvignon 2006,"₩283,554",4.4,"['width: 15%; left: 84.7471%;', 'width: 15%; l..."
1,Elevage 2008,"₩360,000",4.4,"['width: 15%; left: 84.3588%;', 'width: 15%; l..."
2,Colli Della Toscana Centrale Cignale 2006,"₩137,481",4.4,"['width: 15%; left: 51.7868%;', 'width: 15%; l..."
3,GR Reserve Shiraz 2006,"₩171,421",4.4,"['width: 15%; left: 85%;', 'width: 15%; left: ..."
4,Cabernet Sauvignon 2012,"₩280,000",4.4,"['width: 15%; left: 85%;', 'width: 15%; left: ..."


In [963]:
df = pd.merge(df1, df2, how='outer')

In [964]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 816 entries, 0 to 815
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    816 non-null    object 
 1   price   816 non-null    object 
 2   score   816 non-null    float64
 3   temp    816 non-null    object 
dtypes: float64(1), object(3)
memory usage: 31.9+ KB


In [965]:
df_clean=df

In [966]:
df.duplicated().sum()

0

temp가 []인 row를 drop하기

In [967]:
drop_index=df_clean[df_clean.temp=='[]'].index
df_clean=df_clean.drop(drop_index, axis=0)
df_clean.reset_index(drop=True, inplace=True)

전처리 1. price -> 원 표시 떼고 int로 변환

In [968]:
price_row = df_clean.price
df_clean.price = price_row.str.replace(r'[^0-9]', '')
df_clean = df_clean.astype({'price' : 'int'})
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    770 non-null    object 
 1   price   770 non-null    int64  
 2   score   770 non-null    float64
 3   temp    770 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 24.2+ KB


/var/folders/_8/qgs27d456t3f7jzjt6dj6b140000gn/T/ipykernel_21497/1029733992.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_clean.price = price_row.str.replace(r'[^0-9]', '')


전처리 2. temp에 있는 요소들을 body, tannin, sweetness, acidity로 새로운 열 생성

In [969]:
pd.set_option('display.max_columns', None)

In [970]:
temp = df_clean.temp
temp


0      ['width: 15%; left: 66.7221%;', 'width: 15%; l...
1      ['width: 15%; left: 84.3588%;', 'width: 15%; l...
2      ['width: 15%; left: 85%;', 'width: 15%; left: ...
3      ['width: 15%; left: 85%;', 'width: 15%; left: ...
4      ['width: 15%; left: 74.4108%;', 'width: 15%; l...
                             ...                        
765    ['width: 15%; left: 80.7168%;', 'width: 15%; l...
766    ['width: 15%; left: 20.3717%;', 'width: 15%; l...
767    ['width: 15%; left: 84.8789%;', 'width: 15%; l...
768    ['width: 15%; left: 74.1519%;', 'width: 15%; l...
769    ['width: 15%; left: 65.4%;', 'width: 15%; left...
Name: temp, Length: 770, dtype: object

In [971]:
temp = temp.str.extractall(r'left:\s*([\d.]+)%').groupby(level=0).apply(lambda x: ','.join(x[0]))
temp.head()

0    66.7221,58.5902,7.71501,62.9565
1    84.3588,52.4871,4.70766,61.9616
2         85,55.7497,3.97318,55.6561
3         85,55.7497,3.97318,55.6561
4      74.4108,35.6963,16.098,45.685
dtype: object

In [972]:
temp = temp.str.split(',')
temp.head(200)

0      [66.7221, 58.5902, 7.71501, 62.9565]
1      [84.3588, 52.4871, 4.70766, 61.9616]
2           [85, 55.7497, 3.97318, 55.6561]
3           [85, 55.7497, 3.97318, 55.6561]
4        [74.4108, 35.6963, 16.098, 45.685]
                       ...                 
195     [84.974, 51.0283, 6.96718, 53.0525]
196         [85, 50.8381, 8.07094, 61.8649]
197     [66.9438, 23.125, 42.4791, 15.1371]
198     [84.8858, 48.3958, 9.6777, 49.7521]
199    [62.7489, 54.8397, 8.86939, 60.7391]
Length: 200, dtype: object

In [973]:
temp_df = pd.DataFrame(columns=['body', 'tannin', 'sweetness', 'acidity'])


In [974]:
for row in temp:
    try:
        temp_df = temp_df.append({'body' : float(row[0]), 'tannin' : float(row[1]), 'sweetness': float(row[2]), 'acidity' : float(row[3])}, ignore_index=True)
    except IndexError:
        temp_df = temp_df.append({'body' : float(row[0]), 'tannin' : float(row[1]), 'sweetness': float(row[2]), 'acidity' : 0}, ignore_index=True)


/var/folders/_8/qgs27d456t3f7jzjt6dj6b140000gn/T/ipykernel_21497/308663937.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append({'body' : float(row[0]), 'tannin' : float(row[1]), 'sweetness': float(row[2]), 'acidity' : float(row[3])}, ignore_index=True)
/var/folders/_8/qgs27d456t3f7jzjt6dj6b140000gn/T/ipykernel_21497/308663937.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append({'body' : float(row[0]), 'tannin' : float(row[1]), 'sweetness': float(row[2]), 'acidity' : float(row[3])}, ignore_index=True)
/var/folders/_8/qgs27d456t3f7jzjt6dj6b140000gn/T/ipykernel_21497/308663937.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append({'body' : float(row[0]), 

In [975]:
acidity_idx = temp_df[temp_df['acidity'] == 0].index

In [976]:
temp_df.astype('float')

,body,tannin,sweetness,acidity
0,66.7221,58.5902,7.71501,62.9565
1,84.3588,52.4871,4.70766,61.9616
2,85.0000,55.7497,3.97318,55.6561
3,85.0000,55.7497,3.97318,55.6561
4,74.4108,35.6963,16.09800,45.6850
...,...,...,...,...
765,80.7168,47.4183,15.32180,43.8725
766,20.3717,20.1612,2.73472,60.4110
767,84.8789,39.9296,21.68380,40.5364
768,74.1519,46.5191,5.87261,44.4097


In [977]:
# acidity가 없는 데이터는 평균으로 대체
avg_acid = temp_df['acidity'].mean()
for i in acidity_idx:
    temp_df.loc[i]['acidity'] = avg_acid

temp_df.loc[acidity_idx]    

,body,tannin,sweetness,acidity
88,55.09530,46.9453,49.4456,50.655229
95,85.00000,56.1381,64.5698,50.655229
100,39.85370,32.4624,66.9126,50.655229
259,43.95340,24.5168,55.1691,50.655229
268,28.62500,18.7542,69.3708,50.655229
341,62.96940,55.3950,65.1692,50.655229
370,55.44620,37.1715,37.7602,50.655229
422,65.77810,39.4920,46.6555,50.655229
466,67.75000,74.2857,68.3036,50.655229
576,65.97960,60.2619,67.1905,50.655229


전처리 3. body, tannin, sweetness, acidity를 1~5까지로 구간 나누기

In [978]:
def make_1_to_5 (column):
    for i in range(len(temp_df)):
        x = temp_df.loc[i][column] 
        if x <= 20:
            x = 1
        elif x <= 40:
            x = 2
        elif x <= 60:
            x = 3
        elif x <= 80:
            x = 4
        else:
            x = 5
        temp_df.loc[i][column] = x
    return temp_df[column]

for col in ['body', 'tannin', 'sweetness', 'acidity']:
    temp_df[col] = make_1_to_5(col)

temp_df.head()


,body,tannin,sweetness,acidity
0,4.0,3.0,1.0,4.0
1,5.0,3.0,1.0,4.0
2,5.0,3.0,1.0,3.0
3,5.0,3.0,1.0,3.0
4,4.0,2.0,1.0,3.0


In [979]:
temp_df = temp_df.astype('int')

In [980]:
df_clean = df_clean.drop('temp', axis=1)
df_clean = pd.concat([df_clean, temp_df], axis=1)

In [981]:
df_clean.head()

,name,price,score,body,tannin,sweetness,acidity
0,Larose Red Blend 2019,286618,4.7,4,3,1,4
1,Elevage 2000,360000,4.7,5,3,1,4
2,Spring Mountain Cabernet Sauvignon 2011,500000,4.7,5,3,1,3
3,Spring Mountain Cabernet Sauvignon 2016,500000,4.6,5,3,1,3
4,Kai Carmenère 2008,500000,4.6,4,2,1,3


In [982]:
df_clean = df_clean.drop_duplicates()
df_clean = df_clean.reset_index(drop=True)
df_clean

,name,price,score,body,tannin,sweetness,acidity
0,Larose Red Blend 2019,286618,4.7,4,3,1,4
1,Elevage 2000,360000,4.7,5,3,1,4
2,Spring Mountain Cabernet Sauvignon 2011,500000,4.7,5,3,1,3
3,Spring Mountain Cabernet Sauvignon 2016,500000,4.6,5,3,1,3
4,Kai Carmenère 2008,500000,4.6,4,2,1,3
...,...,...,...,...,...,...,...
763,HRB Shiraz 2008,90000,3.8,5,3,1,3
764,Pinot Noir 2018,33568,3.8,2,2,1,4
765,Rob & Les Shiraz 2018,75000,3.8,5,2,2,3
766,Reserva Cabernet Sauvignon 2016,80000,3.8,4,3,1,3


price 구간 나누기

In [986]:
def price_split(x):
    price_x = None
    if x <= 50000:
        price_x = 1
    elif x <= 100000:
        price_x = 2
    elif x <= 150000:
        price_x = 3
    else:
        price_x = 4
    return price_x

df_clean['price'] = df_clean['price'].apply(price_split)

csv 파일 저장

In [988]:
df_clean.to_csv(base_dir + '/db_data.csv', index=False)

db 파일 만들기 (sqlite3)

In [989]:
import psycopg2
import csv

host = 'ziggy.db.elephantsql.com'
user = 'nxsyeowh'
password = 'TgTJdb-06koQQ569eLOr_u2gSqKnwVIC'
database = 'nxsyeowh'

In [996]:
conn = psycopg2.connect(
    host=host,
    user=user,
    password=password,
    database=database)

cur = conn.cursor()

cur.execute("DROP TABLE IF EXISTS wine;")
cur.execute("""CREATE TABLE IF NOT EXISTS wine(
                name VARCHAR(200),
                price INTEGER,
                score FLOAT,
                body INTEGER,
                tannin INTEGER,
                sweetness INTEGER,
                acidity INTEGER
);""")

with open(base_dir +'/db_data.csv', 'r', newline='') as f:
    r = csv.reader(f)
    next(r)
    for row in r:
        cur.execute("""INSERT INTO wine VALUES(
                    %s, %s, %s, %s, %s, %s, %s);""", tuple(row))

conn.commit()
conn.close()
cur.close()
